In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 299 (delta 2), reused 4 (delta 1), pack-reused 293
Receiving objects: 100% (299/299), 72.15 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Checking out files: 100% (47/47), done.


In [2]:
cd first-order-model

/content/first-order-model


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
ls /content/gdrive/My\ Drive/DeepFake/

Isee.mov  themiddleaged.jpeg  theyoung.jpeg  vox-adv-cpk.pth.tar


In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/DeepFake/themiddleaged.jpeg')
driving_video = imageio.mimread('/content/gdrive/My Drive/DeepFake/Isee.mov')
#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/DeepFake/vox-adv-cpk.pth.tar')

In [8]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

100%|██████████| 154/154 [00:29<00:00,  5.24it/s]


In [9]:

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display(source_image, driving_video, predictions).to_html5_video())